## Deliverable 2. Create a Customer Travel Destinations Map.

In [102]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [103]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("cities.csv")
city_data_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'cities.csv'

In [83]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 120


In [88]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,2021-04-09 20:08:45,-23.1203,-134.9692,77.63,71,63,9.42,broken clouds
2,2,Vaini,TO,2021-04-09 20:08:46,-21.2000,-175.2000,77.00,78,40,4.61,scattered clouds
4,4,Butaritari,KI,2021-04-09 20:08:47,3.0707,172.7902,82.38,78,100,16.11,light rain
8,8,Faanui,PF,2021-04-09 20:08:48,-16.4833,-151.7500,81.66,70,100,9.33,overcast clouds
10,10,Albany,US,2021-04-09 20:05:09,42.6001,-73.9662,69.80,35,75,13.80,broken clouds
15,15,Malartic,CA,2021-04-09 20:08:51,48.1335,-78.1328,71.60,37,75,14.97,broken clouds
16,16,Atuona,PF,2021-04-09 20:08:51,-9.8000,-139.0333,81.36,73,43,17.36,scattered clouds
18,18,Labuan,MY,2021-04-09 20:08:52,5.2767,115.2417,78.80,94,20,3.22,few clouds
22,22,Gizo,SB,2021-04-09 20:08:54,-8.1030,156.8419,81.07,81,100,14.03,moderate rain
24,24,Goderich,CA,2021-04-09 20:08:55,43.7501,-81.7165,68.00,52,9,17.27,clear sky


In [89]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                360
City                   360
Country                355
Date                   360
Lat                    360
Lng                    360
Max Temp               360
Humidity               360
Cloudiness             360
Wind Speed             360
Current Description    360
dtype: int64

In [90]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
print (preferred_cities_df)

     City_ID                    City Country                 Date      Lat  \
0          0                 Rikitea      PF  2021-04-09 20:08:45 -23.1203   
2          2                   Vaini      TO  2021-04-09 20:08:46 -21.2000   
4          4              Butaritari      KI  2021-04-09 20:08:47   3.0707   
8          8                  Faanui      PF  2021-04-09 20:08:48 -16.4833   
10        10                  Albany      US  2021-04-09 20:05:09  42.6001   
..       ...                     ...     ...                  ...      ...   
696      696      Concepcion Del Oro      MX  2021-04-09 20:13:30  24.6333   
700      700  Santa Maria Da Vitoria      BR  2021-04-09 20:13:32 -13.3947   
702      702                  Nobres      BR  2021-04-09 20:13:33 -14.7203   
705      705                Waingapu      ID  2021-04-09 20:13:34  -9.6567   
706      706                  Niamey      NE  2021-04-09 20:13:34  13.5137   

          Lng  Max Temp  Humidity  Cloudiness  Wind Speed Curre

In [91]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.63,broken clouds,-23.1203,-134.9692,
2,Vaini,TO,77.00,scattered clouds,-21.2000,-175.2000,
4,Butaritari,KI,82.38,light rain,3.0707,172.7902,
8,Faanui,PF,81.66,overcast clouds,-16.4833,-151.7500,
10,Albany,US,69.80,broken clouds,42.6001,-73.9662,
15,Malartic,CA,71.60,broken clouds,48.1335,-78.1328,
16,Atuona,PF,81.36,scattered clouds,-9.8000,-139.0333,
18,Labuan,MY,78.80,few clouds,5.2767,115.2417,
22,Gizo,SB,81.07,moderate rain,-8.1030,156.8419,
24,Goderich,CA,68.00,clear sky,43.7501,-81.7165,


In [92]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
    
}
# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
# 6d. Set up the base URL for the Google Directions API to get JSON data. 
   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #6e Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
         
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
                                                                                                                 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [93]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.63,broken clouds,-23.1203,-134.9692,Pension Maro'i
2,Vaini,TO,77.00,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
4,Butaritari,KI,82.38,light rain,3.0707,172.7902,Isles Sunset Lodge
8,Faanui,PF,81.66,overcast clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
10,Albany,US,69.80,broken clouds,42.6001,-73.9662,
...,...,...,...,...,...,...,...
696,Concepcion Del Oro,MX,81.01,clear sky,24.6333,-101.4167,HOTEL HACIENDA SAN JAVIER
700,Santa Maria Da Vitoria,BR,89.31,overcast clouds,-13.3947,-44.1886,Pousada Rio Current
702,Nobres,BR,84.09,broken clouds,-14.7203,-56.3275,Hotel Portal Nobres
705,Waingapu,ID,77.02,clear sky,-9.6567,120.2641,Pondok Wisata Elim


In [101]:
# 8a. Create the output File (CSV)
output_data_file = "Hotels.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [104]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Descriptio</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [105]:
# 11a. Add a marker layer for each city to the map.

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure.
fig

Figure(layout=FigureLayout(height='420px'))